In [ ]:
import cv2
import numpy as np
import time
from matplotlib import pyplot as plt

# Set the path for the input image
img_name = 'image.jpg'
img = cv2.imread(img_name)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Load YOLO model weights and configuration
net = cv2.dnn.readNetFromDarknet('yolov3-custom.cfg', 'yolov3-custom.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

# Load the classes from the dataset (make sure 'person' is correctly labeled in your 'crowd.names')
classes = open('crowd.names').read().strip().split('\n')
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

# Determine only the output layer names that we need from YOLO
ln = [net.getLayerNames()[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Convert image to blob
blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
outputs = net.forward(ln)

boxes = []
confidences = []
classIDs = []
h, w = img.shape[:2]

# Initialize counter for people
people_count = 0

# Process each of the outputs from the network
for output in outputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > 0.5:
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

            # Count if the detected class is 'person'
            if classes[classID] == 'person':
                people_count += 1

# Perform non maximum suppression to eliminate redundant overlapping boxes with lower confidences
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
if len(indices) > 0:
    for i in indices.flatten():
        (x, y, w, h) = boxes[i]
        color = [int(c) for c in colors[classIDs[i]]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
        cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Output the result
print(f"Number of people detected: {people_count}")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Detected Objects')
plt.show()
